## PReader (old)
I couldn't make the PReader class be just a recursive algorithm. For long strings, this would mean that the stack would get very big. Some sort of buffer would have to prevent the stack from getting too big while also keeping the right order of characters. 

I created a char array from the string and a cursor. The cursor is a position indicator for which character is being read. In addition we would have to check for certain trigger characters, like operators for starting a new operation, parentheses to capture the operation in, and letters that would be variables. Taking this into account, we should also create a stack for read operations up until the current character. We would then interpret the operator when a closing parenthesis occurs.

## Testing arena below

In [11]:
from proposition import *
from proposition_operator import *
from proposition_parsing import *
from proposition_export import *
from hashing import *
from truthtable import *

and_operator = OperatorFactory.get_operator('&')
or_operator = OperatorFactory.get_operator('|')
impl_operator = OperatorFactory.get_operator('>')
biimpl_operator = OperatorFactory.get_operator('=')
not_operator = OperatorFactory.get_operator('~')

prop = CompoundProposition(or_operator,
    CompoundProposition(or_operator, Variable('A'), Variable('B')),
    Variable('C')
)

print(prop)
print(prop.infix())

|(|(A,B),C)
((A ∨ B) ∨ C)


In [12]:
truthtable = TruthTable(prop)

print(truthtable)

A B C result
0 0 0 0
0 0 1 1
0 1 0 1
0 1 1 1
1 0 0 1
1 0 1 1
1 1 0 1
1 1 1 1



In [13]:
simplified_truthtable = truthtable.simplify()

print(simplified_truthtable)

A B C result
0 0 0 0
* * 1 1
* 1 * 1
1 * * 1



In [14]:
print(simplified_truthtable.get_binary_string())
print(simplified_truthtable.get_hash())

1110
E


In [15]:
prop = PropositionParser("&(~(A),>(>(C,|(D,E)),~(B)))").read()
print(prop.infix())

truthtable = TruthTable(prop)
print(truthtable)

(¬A ∧ ((C => (D ∨ E)) => ¬B))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 0 0 0 0 0
1 0 0 0 1 0
1 0 0 1 0 0
1 0 0 1 1 0
1 0 1 0 0 0
1 0 1 0 1 0
1 0 1 1 0 0
1 0 1 1 1 0
1 1 0 0 0 0
1 1 0 0 1 0
1 1 0 1 0 0
1 1 0 1 1 0
1 1 1 0 0 0
1 1 1 0 1 0
1 1 1 1 0 0
1 1 1 1 1 0



In [16]:
truthtable.simplify()
print(truthtable)
print(f"hash: {truthtable.get_hash()}")

A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF


In [17]:
dnf_prop = truthtable.dnf()
print(dnf_prop.infix())

dnf_truthtable = TruthTable(dnf_prop)
dnf_truthtable.simplify()
print(dnf_truthtable)
print(f"hash: {dnf_truthtable.get_hash()}")

((¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ E) ∨ (¬A ∧ B ∧ C ∧ ¬D ∧ ¬E))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF
